<a href="https://colab.research.google.com/github/evildead23151/Practical-Deep-Learning/blob/main/LearningPyTorch%7BQuickStart%7D.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor

We are going to learn pyTorch by a sample set of data FasionMNIST

1. DATA

In [4]:
#Getting data from datasets
training_data = datasets.FashionMNIST(
    root="data",
    train=True,
    download=True,
    transform=ToTensor(),
)
#Getting data for testing later
test_data = datasets.FashionMNIST(
    root="data",
    train=False,
    download=True,
    transform=ToTensor(),
)

100%|██████████| 26.4M/26.4M [00:02<00:00, 10.9MB/s]
100%|██████████| 29.5k/29.5k [00:00<00:00, 208kB/s]
100%|██████████| 4.42M/4.42M [00:01<00:00, 3.91MB/s]
100%|██████████| 5.15k/5.15k [00:00<00:00, 9.34MB/s]


In [5]:
batch_size = 64
train_dataloader = DataLoader(training_data, batch_size=batch_size)
test_dataloader = DataLoader(test_data, batch_size=batch_size)
for X, y in test_dataloader:
  print(f"shape of X [N, C, H, W]:{X.shape}")
  print(f"shape of y:{y.shape}{y.dtype}")

shape of X [N, C, H, W]:torch.Size([64, 1, 28, 28])
shape of y:torch.Size([64])torch.int64
shape of X [N, C, H, W]:torch.Size([64, 1, 28, 28])
shape of y:torch.Size([64])torch.int64
shape of X [N, C, H, W]:torch.Size([64, 1, 28, 28])
shape of y:torch.Size([64])torch.int64
shape of X [N, C, H, W]:torch.Size([64, 1, 28, 28])
shape of y:torch.Size([64])torch.int64
shape of X [N, C, H, W]:torch.Size([64, 1, 28, 28])
shape of y:torch.Size([64])torch.int64
shape of X [N, C, H, W]:torch.Size([64, 1, 28, 28])
shape of y:torch.Size([64])torch.int64
shape of X [N, C, H, W]:torch.Size([64, 1, 28, 28])
shape of y:torch.Size([64])torch.int64
shape of X [N, C, H, W]:torch.Size([64, 1, 28, 28])
shape of y:torch.Size([64])torch.int64
shape of X [N, C, H, W]:torch.Size([64, 1, 28, 28])
shape of y:torch.Size([64])torch.int64
shape of X [N, C, H, W]:torch.Size([64, 1, 28, 28])
shape of y:torch.Size([64])torch.int64
shape of X [N, C, H, W]:torch.Size([64, 1, 28, 28])
shape of y:torch.Size([64])torch.int64

2. CREATING MODELS

In [6]:
device = torch.accelerator.current_accelerator().type if torch.accelerator.is_available() else "cpu"
print(f"Using {device} device")

class NeuralNetwork(nn.Module):
  def __init__(self):
    super().__init__()
    self.flatten = nn.Flatten()
    self.linear_relu_stack = nn.Sequential(
        nn.Linear(28*28, 512),
        nn.ReLU(),
        nn.Linear(512, 512),
        nn.ReLU(),
        nn.Linear(512, 10)
    )
  def forward(self, x):
    x = self.flatten(x)
    logits = self.linear_relu_stack(x)
    return logits
model = NeuralNetwork().to(device)
print(model)

Using cpu device
NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


3. Optimizing the Model Parameters

3.1. to train a model, we need a loss function and an optimizer.

In [7]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr = 1e-3)

3.2

In [8]:
def train(dataloader, mode, loss_fn, optimizer):
  size = len(dataloader.dataset)
  model.train()
  for batch, (X, y) in enumerate(dataloader):
    X, y = X.to(device), y.to(device)

    pred = model(X)
    loss = loss_fn(pred, y)
    #Backpropagation
    loss.backward()
    optimizer.step()
    optimizer.zero_grad()

    if batch%100 == 0:
      loss, current = loss.item(), (batch +1)*len(X)
      print(f"loss: {loss:>7f} [{current:>5d}/{size:>5d}]")


In [16]:
def test(dataloader, model, loss_fn):
  size = len(dataloader.dataset)
  num_batches = len(dataloader)
  model.eval()
  test_loss, correct = 0, 0
  with torch.no_grad():
    for X, y in dataloader:
      X, y = X.to(device), y.to(device)
      pred = model(X)
      test_loss += loss_fn(pred, y).item()
      correct += (pred.argmax(1) == y).type(torch.float).sum().item()
  test_loss /= num_batches
  correct /= size
  print(f"Test Error: \n Accuracy: {(100*correct):.1f}%, Avg loss: {test_loss:.8f} \n")

In [17]:
epochs = 5
for t in range(epochs):
  print(f"Epoch {t+1}\n--------------------")
  train(train_dataloader, model, loss_fn, optimizer)
  test(test_dataloader, model, loss_fn)
print("Done!")

Epoch 1
--------------------
loss: 1.350026 [   64/60000]
loss: 1.341615 [ 6464/60000]
loss: 1.165470 [12864/60000]
loss: 1.275508 [19264/60000]
loss: 1.161828 [25664/60000]
loss: 1.168739 [32064/60000]
loss: 1.192387 [38464/60000]
loss: 1.125712 [44864/60000]
loss: 1.170092 [51264/60000]
loss: 1.083625 [57664/60000]
Test Error: 
 Accuracy: 64.1%, Avg loss: 1.10625391 

Epoch 2
--------------------
loss: 1.171679 [   64/60000]
loss: 1.182287 [ 6464/60000]
loss: 0.991855 [12864/60000]
loss: 1.129894 [19264/60000]
loss: 1.011682 [25664/60000]
loss: 1.029839 [32064/60000]
loss: 1.065961 [38464/60000]
loss: 1.007274 [44864/60000]
loss: 1.051863 [51264/60000]
loss: 0.977406 [57664/60000]
Test Error: 
 Accuracy: 65.1%, Avg loss: 0.99460830 

Epoch 3
--------------------
loss: 1.048932 [   64/60000]
loss: 1.079377 [ 6464/60000]
loss: 0.874256 [12864/60000]
loss: 1.033719 [19264/60000]
loss: 0.916929 [25664/60000]
loss: 0.932214 [32064/60000]
loss: 0.984272 [38464/60000]
loss: 0.930926 [44864/

4. Saving Models

In [18]:
torch.save(model.state_dict(), "model.pth")
print("Saved PyTorch model state to model.pth")

Saved PyTorch model state to model.pth


5. Loading Models

In [20]:
model = NeuralNetwork().to(device)
model.load_state_dict(torch.load("model.pth", weights_only= True))

<All keys matched successfully>

Model can now be used to make preidctions.

In [21]:
classes = [
    "T-shirt/top",
    "Trouser",
    "Pullover",
    "Dress",
    "Coat",
    "Sandal",
    "Shirt",
    "Sneaker",
    "Bag",
    "Ankle boot",
]
model.eval()
x, y = test_data[0][0], test_data[0][1]
with torch.no_grad():
  x = x.to(device)
  pred = model(x)
  predicted, actual = classes[pred[0].argmax(0)], classes[y]
  print(f'Predicted: "{predicted}", Actual:"{actual}"')

Predicted: "Ankle boot", Actual:"Ankle boot"
